## Preparing a dataset for centre back model
Aim is to collect a dataset on match lineups (defenders specifically) and match stats, specifically xG, goals and also touches in box / final 3rd.

### 1. Set up / dependencies

In [25]:
from statsbombpy import sb
import numpy as np
import pandas as pd

### 2. Read in data

In [27]:
# get the competition / season_ids
comps = sb.competitions()
comps = comps[comps.competition_id == 37]
comps.season_id

credentials were not supplied. open data access only


15    90
16    42
17     4
Name: season_id, dtype: int64

In [32]:
# get all the match ids
matches1 = sb.matches(competition_id=37, season_id=4)
matches2 = sb.matches(competition_id=37, season_id=42)
matches3 = sb.matches(competition_id=37, season_id=90)
matches = pd.concat([matches1, matches2, matches3])
match_ids = matches.match_id.values

credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only


In [46]:
# get the lineup data
lineups = pd.DataFrame()
for m in match_ids:
    lineup = sb.lineups(match_id=m)
    teams = list(lineup.keys())
    team1 = teams[0]
    team2 = teams[1]
    lineup1 = pd.DataFrame(lineup[team1])
    lineup2 = pd.DataFrame(lineup[team2])
    lineup1["team"] = team1
    lineup2["team"] = team2  
    lineup1["match_id"] = m
    lineup2["match_id"] = m
    lineups = pd.concat([lineups, lineup1, lineup2])

credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data acces

credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data acces

credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only


In [81]:
# extract key info to new df columns
lineups["position"] = lineups["positions"].apply(lambda x: x[0]["position"] if len(x) > 0 else None)
lineups["start_time"] = lineups["positions"].apply(lambda x: x[0]["from"] if len(x) > 0 else None)
lineups["end_time"] = lineups["positions"].apply(lambda x: x[0]["to"] if len(x) > 0 else None)

In [92]:
# only really interested in centre backs
print(lineups.position.unique())
cbs = ["Left Center Back", "Right Center Back", "Center Back"]
cb_lineups = lineups[lineups.position.isin(cbs)]

['Left Center Back' 'Left Center Forward' 'Left Center Midfield'
 'Goalkeeper' 'Right Center Forward' 'Right Center Back' 'Center Midfield'
 'Center Back' 'Right Center Midfield' 'Left Midfield' 'Center Forward'
 'Right Midfield' 'Left Wing' 'Right Back' 'Left Back' 'Right Wing'
 'Center Attacking Midfield' None 'Left Defensive Midfield'
 'Right Defensive Midfield' 'Center Defensive Midfield'
 'Left Attacking Midfield' 'Right Attacking Midfield' 'Secondary Striker'
 'Right Wing Back' 'Left Wing Back']


In [93]:
cb_lineups.shape

(1476, 12)

In [108]:
# get event data related to matches
events = sb.events(match_id=match_ids[0])

credentials were not supplied. open data access only


In [109]:
events.shape

(3793, 35)

In [110]:
events.sample(5)

,bad_behaviour,ball_receipt,ball_recovery,block,carry,counterpress,dribble,duel,duration,foul_committed,...,possession_team,related_events,second,shot,substitution,tactics,team,timestamp,type,under_pressure
1017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Manchester City WFC,"[30041b72-609a-4bfb-bb5a-2ee981b8fdbb, 30dca49...",57,NaN,NaN,NaN,Manchester City WFC,00:01:57.793,Ball Receipt*,True
2015,NaN,NaN,NaN,NaN,"{'end_location': [35.0, 41.0]}",NaN,NaN,NaN,1.213,NaN,...,Manchester City WFC,"[c4ca8edd-5be4-4b6e-bfd8-45a1d00656d8, dfe747e...",27,NaN,NaN,NaN,Manchester City WFC,00:09:27.700,Carry,NaN
3388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Chelsea FCW,NaN,34,NaN,NaN,NaN,Chelsea FCW,00:16:34.838,Ball Recovery,NaN
2316,NaN,NaN,NaN,NaN,"{'end_location': [68.0, 13.0]}",NaN,NaN,NaN,4.320,NaN,...,Chelsea FCW,"[09f33b45-e25a-4f4a-a04f-63a530cff412, 9e278a3...",27,NaN,NaN,NaN,Chelsea FCW,00:43:27.100,Carry,True
738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.040,NaN,...,Chelsea FCW,[eeb4f585-3ec0-4bee-8ef8-909ee1f8df41],47,NaN,NaN,NaN,Chelsea FCW,00:19:47.998,Pass,NaN


In [98]:
events.columns

Index(['bad_behaviour', 'ball_receipt', 'ball_recovery', 'block', 'carry',
       'counterpress', 'dribble', 'duel', 'duration', 'foul_committed',
       'foul_won', 'goalkeeper', 'id', 'index', 'injury_stoppage',
       'interception', 'location', 'match_id', 'minute', 'pass', 'period',
       'play_pattern', 'player', 'position', 'possession', 'possession_team',
       'related_events', 'second', 'shot', 'substitution', 'tactics', 'team',
       'timestamp', 'type', 'under_pressure'],
      dtype='object')

In [120]:
# get shot data
shots = events[events.shot.notnull()]
shots = shots[["match_id", "id", "player", "minute", "possession_team", "shot"]]
shots["xG"] = shots["shot"].apply(lambda x: x["statsbomb_xg"])

In [132]:
# get possession data
possession = events[["match_id", "possession_team", "duration", "type", "location", "minute"]]

### 3. Clean data
The clean dataset needs to contain each centre back combination for each game for each team, with which minutes they played and the possession / xG stats for those minutes

In [179]:
cb_lineups["start_minute"] = cb_lineups["start_time"].apply(lambda x: int(str(x).split(":")[0]))
cb_lineups["end_minute"] = cb_lineups["end_time"].apply(lambda x: int(str(x).split(":")[0]) if x is not None else None)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [180]:
cb_lineups.head(2)

,player_id,player_name,player_nickname,jersey_number,country,cards,positions,team,match_id,position,start_time,end_time,start_minute,end_minute
0,4633,Magdalena Lilly Eriksson,None,16,Sweden,[],"[{'position_id': 5, 'position': 'Left Center B...",Chelsea FCW,7298,Left Center Back,00:00,None,0,NaN
5,4642,Millie Bright,None,4,England,[],"[{'position_id': 3, 'position': 'Right Center ...",Chelsea FCW,7298,Right Center Back,00:00,31:37,0,31.0


In [240]:
# Get the lineups for each match first
player_ids = []
starts = []
ends = []
matches = []
teams = []

# get unique match / team combos
mts = set(list(zip(cb_lineups.match_id.values, cb_lineups.team.values))[:20])

# loop through match-teams and compute different lineup combinations
for m, t in mts:
    # get basic info / num lineup changes
    match_team_df = cb_lineups[(cb_lineups.match_id == m) & (cb_lineups.team == t)]
    num_lineups = match_team_df[match_team_df.end_minute.notnull()].shape[0] + 1
    matches.extend([m] * num_lineups)
    teams.extend([t] * num_lineups)
    
    # get first lineup
    starts.append(0)
    ends.append(min(match_team_df["end_minute"]))
    current_lineup = list(match_team_df[match_team_df.start_minute == 0].player_id.values)
    player_ids.append(current_lineup)
    
    # get subsequent lineups
    if num_lineups != 1:
        changes = [x for x in match_team_df.start_minute.values if x > 0]
        change_ends = [x for x in match_team_df.end_minute.values if x > 0]
        changes.extend(change_ends)

In [241]:
changes

[31.0]

In [227]:
change_ends

[nan, 31.0, nan]

In [217]:
teams

['Manchester City WFC',
 'Manchester City WFC',
 'Birmingham City WFC',
 'Chelsea FCW',
 'Arsenal WFC',
 'Birmingham City WFC',
 'Yeovil Town LFC',
 'Yeovil Town LFC',
 'Yeovil Town LFC',
 'Chelsea FCW',
 'Chelsea FCW',
 'Yeovil Town LFC',
 'Brighton & Hove Albion WFC',
 'Manchester City WFC']

In [218]:
player_ids

[[4648, 17524],
 [19502, 19503],
 [4633, 4642],
 [10666, 18999],
 [19502, 19503],
 [15715, 15717],
 [4633, 4642, 4657],
 [15712, 15717],
 [16394, 16395],
 [10185, 17524]]

In [219]:
starts

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [220]:
ends

[33.0, nan, nan, nan, nan, 4.0, nan, nan, nan, nan]

In [207]:
m1 = 10
m = list(mts)[m1][0]
t = list(mts)[m1][1]

IndexError: list index out of range

In [187]:
match_team_df = cb_lineups[(cb_lineups.match_id == m) & (cb_lineups.team == t)]

In [189]:
match_team_df[match_team_df.end_minute.notnull()].shape[0]

0

In [188]:
match_team_df

,player_id,player_name,player_nickname,jersey_number,country,cards,positions,team,match_id,position,start_time,end_time,start_minute,end_minute
1,15618,Jasmine Matthews,None,4,England,[],"[{'position_id': 4, 'position': 'Center Back',...",Bristol City WFC,3775625,Center Back,00:00,None,0,NaN
6,24922,Florence Allen,None,12,England,[],"[{'position_id': 3, 'position': 'Right Center ...",Bristol City WFC,3775625,Right Center Back,00:00,None,0,NaN
13,46741,Naomi Layzell,None,25,England,[],"[{'position_id': 5, 'position': 'Left Center B...",Bristol City WFC,3775625,Left Center Back,00:00,None,0,NaN


In [137]:
possession.sample(2)

,match_id,possession_team,duration,type,location,minute
2608,7298,Manchester City WFC,3.24,Carry,"[71.0, 9.0]",72
1752,7298,Manchester City WFC,NaN,Ball Receipt*,"[76.0, 9.0]",72


In [155]:
cb_lineups[cb_lineups.match_id == match_ids[132]]

,player_id,player_name,player_nickname,jersey_number,country,cards,positions,team,match_id,position,start_time,end_time
8,10185,Stephanie Houghton,None,6,England,[],"[{'position_id': 3, 'position': 'Right Center ...",Manchester City WFC,2275092,Right Center Back,00:00,None
10,15554,Gemma Bonner,None,4,England,[],"[{'position_id': 5, 'position': 'Left Center B...",Manchester City WFC,2275092,Left Center Back,00:00,None
2,15569,Kerys Harrop,None,6,England,[],"[{'position_id': 5, 'position': 'Left Center B...",Birmingham City WFC,2275092,Left Center Back,74:11,None
7,19592,Harriet Scott,None,3,Ireland,[],"[{'position_id': 3, 'position': 'Right Center ...",Birmingham City WFC,2275092,Right Center Back,00:00,None
13,31568,Rebecca Holloway,None,25,Northern Ireland,[],"[{'position_id': 5, 'position': 'Left Center B...",Birmingham City WFC,2275092,Left Center Back,00:00,74:11


In [140]:
shots.sample(2)

,match_id,id,player,minute,possession_team,shot,xG
3535,7298,23d74ba8-6e5d-4855-8b3a-81b310e9d35a,Jill Scott,26,Manchester City WFC,"{'statsbomb_xg': 0.08011432, 'end_location': [...",0.080114
3537,7298,ea3833bf-1f02-4fba-9b27-8a1ebc28d732,Ramona Bachmann,39,Chelsea FCW,"{'statsbomb_xg': 0.024449918, 'end_location': ...",0.024450
